In [12]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# from huggingface_hub import notebook_login

# # Запустите это и вставьте ваш токен в появившееся поле
# notebook_login()

In [13]:
import os
from pathlib import Path

folder_path = "/content/drive/MyDrive/Colab Notebooks/qwen_model"
print(f"Folder exists: {os.path.exists(folder_path)}")
print(f"Is directory: {os.path.isdir(folder_path)}")

Folder exists: True
Is directory: True


In [16]:
# from pathlib import Path

# required_files = [
#     "adapter_config.json",
#     "adapter_model.safetensors",  # или аналогичный файл с весами
#     "tokenizer_config.json",
#     "special_tokens_map.json",
#     "tokenizer.json",
#     "vocab.json",
#     "merges.txt",
#     "config.json",  # если есть
# ]

# for file in required_files:
#     if not (Path(folder_path) / file).exists():
#         print(f"⚠️ Отсутствует обязательный файл: {file}")

⚠️ Отсутствует обязательный файл: config.json


In [ ]:
# from huggingface_hub import HfApi
# api = HfApi()
# api.whoami()  # Должен вернуть ваш username

In [14]:
# from huggingface_hub import HfApi
# from pathlib import Path

# api = HfApi()
# repo_id = "nata2627/angl_detection_tokenizer_qwen"

# # First verify the path
# model_path = "/content/drive/MyDrive/Colab Notebooks/qwen_model"
# if not Path(model_path).is_dir():
#     raise ValueError(f"Model directory not found at: {model_path}")

# # Then upload
# api.upload_folder(
#     folder_path=model_path,
#     repo_id=repo_id,
#     repo_type="model"
# )

CommitInfo(commit_url='https://huggingface.co/nata2627/angl_detection_tokenizer_qwen/commit/c800d9a23e34205485233494aad088dda3638ad9', commit_message='Upload folder using huggingface_hub', commit_description='', oid='c800d9a23e34205485233494aad088dda3638ad9', pr_url=None, repo_url=RepoUrl('https://huggingface.co/nata2627/angl_detection_tokenizer_qwen', endpoint='https://huggingface.co', repo_type='model', repo_id='nata2627/angl_detection_tokenizer_qwen'), pr_revision=None, pr_num=None)

In [ ]:
# !pip install rouge

# **Llama model test**

In [ ]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer
import torch
import pandas as pd

def load_model():
    model = AutoPeftModelForCausalLM.from_pretrained("nata2627/angl_detection_tokenizer_llama")
    tokenizer = AutoTokenizer.from_pretrained("nata2627/angl_detection_tokenizer_llama")

    model = model.to("cuda")
    model.eval()
    return model, tokenizer

def load_test_data(path='test_dataset.csv', n_samples=None):
    """Загружает тестовые данные из CSV файла"""
    df = pd.read_csv(path)
    if n_samples is not None:
        df = df.head(n_samples)
    return list(zip(df['original'], df['replaced']))

In [ ]:
# Загружаем тестовые данные

import pandas as pd
N = 50

# Загрузка датасета
test_data = pd.read_csv('test_dataset.csv')

# Предположим, что у вас два столбца: 'original' и 'reference' — если названия другие, замени их ниже
# Отбор первых двух примеров
originals_df = test_data[['original']].head(N)
references_df = test_data[['replaced']].head(N)

# Преобразование в списки
original = originals_df['original'].tolist()
references = references_df['replaced'].tolist()

# Проверка
print("Originals:", original)
print("References:", references)

Originals: ['По словам женщины, <англицизм>лифты</англицизм> в доме часто ломаются, она уже оставила жалобу в обслуживающей компании и планирует обратиться в правоохранительные органы.', 'Об этом официальный представитель МИД России Мария Захарова сказала во время еженедельного <англицизм>брифинга</англицизм>.', '«Глубина бронирования остается на прежнем уровне, это природное явление, которое очень редко бывает в Эмиратах, поэтому проблема будет решена за один-два дня»,\xa0— подытожил <англицизм>вице-президент</англицизм> РСТ.', 'Сафиуллин сейчас занимает 42-е место в <англицизм>рейтинге</англицизм> ATP (четвертый из россиян).', 'В последний раз Смоленская область подверглась атаке <англицизм>дронов</англицизм> 24 апреля, на топливно-энергетических объектах в Смоленском и Ярцевском районах начались пожары.', 'Тройчук будет руководителем проекта, у которого есть <англицизм>инвестор</англицизм>.', 'Интернет-портал «Чемпионат» 8 апреля сообщил, что голландец скоро возглавит московский <ан

In [ ]:
model, tokenizer = load_model()

In [ ]:
import re

def clean_model_output(text):
    # Удаляем дублирующиеся предложения (если модель повторяется)
    sentences = text.split('.')
    unique_sentences = []
    seen = set()
    for sent in sentences:
        clean_sent = sent.strip()
        if clean_sent and clean_sent not in seen:
            seen.add(clean_sent)
            unique_sentences.append(clean_sent)
    return '. '.join(unique_sentences).strip()

def replace_anglicisms(model, tokenizer, texts):
    """
    Принимает:
        - model: языковая модель (например, LLM)
        - tokenizer: токенизатор, совместимый с моделью
        - texts: список строк (List[str]), каждая из которых содержит текст с англицизмами

    Возвращает:
        - List[str]: список строк без англицизмов
    """
    results = []

    for text in texts:
        formatted_input = f"<|im_start|>user\nТекст: {text}<|im_end|>\n<|im_start|>assistant\n"
        inputs = tokenizer(formatted_input, return_tensors="pt").to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs.input_ids,
                attention_mask=inputs.attention_mask,
                max_new_tokens=256,
                temperature=0.7,
                top_p=0.9,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        full_output = tokenizer.decode(outputs[0], skip_special_tokens=False)

        possible_start_markers = [
            "assistant\n",
            "assistant\nТекст: ",
            "assistant\nТекст:",
            "assistant\nТекст : ",
            "assistant\nТекст :"
        ]

        end_marker = "<|im_end|>"

        for start_marker in possible_start_markers:
            start_idx = full_output.find(start_marker)
            if start_idx != -1:
                start_idx += len(start_marker)
                end_idx = full_output.find(end_marker, start_idx)
                if end_idx != -1:
                    clean_output = full_output[start_idx:end_idx].strip()
                    if clean_output.startswith("Текст:"):
                        clean_output = clean_output[6:].strip()
                    results.append(clean_model_output(clean_output))
                    break
        else:
            # Fallback
            clean_output = full_output.split("assistant")[-1]
            clean_output = clean_output.split("<|im_end|>")[0].strip()
            clean_output = clean_output.replace("<|user|>", "").replace("<|assistant|>", "").replace("<|end|>", "").replace("<|im_start|>", "").replace("<|im_end|>", "").replace("</s>", "").replace("<|end_of_text|>", "").replace("<|end_of|>", "").strip()

            while clean_output.startswith("Текст:"):
                clean_output = clean_output[6:].strip()

            def truncate_to_last_sentence(text):
                """
                Обрезает текст до последнего завершённого предложения (по точке, воскл. или вопрос. знаку).
                """
                sentences = re.split(r'(?<=[.!?])\s+', text)
                if sentences:
                    return ' '.join(sentences[:-1] if len(sentences) > 1 else sentences).strip()
                return text.strip()

            final_output = clean_model_output(clean_output)
            results.append(truncate_to_last_sentence(final_output))

    return results

In [ ]:
predictions = replace_anglicisms(model, tokenizer, original)

In [ ]:
len(predictions)

50

In [ ]:
len(references)

50

In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.tokenize import word_tokenize
from rouge import Rouge

# Загрузка токенизатора
nltk.download('punkt')

# Подготовка
smoothie = SmoothingFunction().method4
rouge = Rouge()

# Таблица метрик по каждому примеру
metrics_table = []

for pred, ref in zip(predictions, references):
    # Убедимся, что длина предсказания и ссылки совпадает
    if len(pred.split()) == 0 or len(ref.split()) == 0:
        continue  # Пропускаем, если предсказание или ссылка пустое

    exact_match = int(pred == ref)

    # Токенизация
    pred_tokens = word_tokenize(pred.lower())
    ref_tokens = [word_tokenize(ref.lower())]
    bleu = sentence_bleu(ref_tokens, pred_tokens, smoothing_function=smoothie)

    try:
        # Вычисление метрик ROUGE
        rouge_score = rouge.get_scores(pred, ref)[0]
    except ValueError:
        # Если возникает ошибка, установим значения метрик в 0
        rouge_score = {
            'rouge-1': {'f': 0},
            'rouge-2': {'f': 0},
            'rouge-l': {'f': 0}
        }

    # Добавление результатов в таблицу
    metrics_table.append({
        'prediction': pred,
        'reference': ref,
        'exact_match': exact_match,
        'bleu': bleu,
        'rouge-1': rouge_score['rouge-1']['f'],
        'rouge-2': rouge_score['rouge-2']['f'],
        'rouge-l': rouge_score['rouge-l']['f'],
        'prediction_length': len(pred.split()),
        'reference_length': len(ref.split())
    })

# Создание DataFrame и сохранение результатов в CSV
df = pd.DataFrame(metrics_table)
df.to_csv('llama_metrics_table.csv', index=False)

# Дополнительно: сводные метрики
summary_metrics = {
    'exact_match_accuracy': df['exact_match'].mean(),
    'bleu_score': df['bleu'].mean(),
    'rouge-1': df['rouge-1'].mean(),
    'rouge-2': df['rouge-2'].mean(),
    'rouge-l': df['rouge-l'].mean(),
    'avg_prediction_length': df['prediction_length'].mean(),
    'avg_reference_length': df['reference_length'].mean()
}

# Преобразуем сводные метрики в DataFrame и сохраняем их в Excel
summary_df = pd.DataFrame([summary_metrics])
summary_df.to_excel('llama_summary_metrics.xlsx', index=False)

# Печать таблицы с метриками
print(df)  # Печать таблицы с метриками
print(summary_metrics)  # Печать сводных метрик

                                           prediction  \
0         |>\nПо словам женщины, уровень в доме часто   
1   Об этом официальный представитель МИД России М...   
2   «Глубина бронирования остается на прежнем уров...   
3   Сафиуллин сейчас занимает 42-е место в оценке ...   
4   В последний раз Смоленская область подверглась...   
5   Тройчук будет руководителем проекта, у которог...   
6   Интернет-портал «Чемпионат» 8 апреля сообщил, ...   
7   Обвиняемыми по делу проходят более 30 человек,...   
8   |>\nАтаки украинских беспилотников на Запорожс...   
9   В то же время, по данным WAtoday, Деспро — стр...   
10  Украина нужна Европейскому союзу и станет хоро...   
11  О взятии под надзор последнего военного ведомс...   
12  Он был главой и совладельцем немецкой сети маг...   
13     Постепенно его распространяют и на другие лица   
14  Главный спортивный наставник «Ростова» Валерий...   
15  На территории двух государств также расположен...   
16  НАТО не пригласит Украину в

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
df

,prediction,reference,exact_match,bleu,rouge-1,rouge-2,rouge-l,prediction_length,reference_length
0,"|>\nПо словам женщины, уровень в доме часто","По словам женщины, подъемник в доме часто лома...",0,0.093687,0.444444,0.296296,0.444444,8,21
1,Об этом официальный представитель МИД России М...,Об этом официальный представитель МИД России М...,0,0.925961,1.000000,1.000000,1.000000,13,13
2,«Глубина бронирования остается на прежнем уров...,«Глубина бронирования остается на прежнем уров...,0,0.867757,0.888889,0.846154,0.888889,27,27
3,Сафиуллин сейчас занимает 42-е место в оценке ...,Сафиуллин сейчас занимает 42-е место в оценке ...,0,0.704805,0.909091,0.800000,0.909091,11,11
4,В последний раз Смоленская область подверглась...,В последний раз Смоленская область подверглась...,0,0.872513,0.950000,0.894737,0.950000,20,20
5,"Тройчук будет руководителем проекта, у которог...","Тройчук станет руководителем проекта, у которо...",0,0.708074,0.888889,0.750000,0.888889,9,9
6,"Интернет-портал «Чемпионат» 8 апреля сообщил, ...","Интернет-портал «Чемпионат» 8 апреля сообщил, ...",0,0.401822,0.956522,0.869565,0.956522,24,11
7,"Обвиняемыми по делу проходят более 30 человек,...","Обвиняемыми по делу проходят более 30 человек,...",0,0.909593,0.954545,0.904762,0.954545,22,22
8,|>\nАтаки украинских беспилотников на Запорожс...,Атаки украинских беспилотников на Запорожскую ...,0,0.878188,0.965517,0.965517,0.965517,16,15
9,"В то же время, по данным WAtoday, Деспро — стр...","В то же время, по данным WAtoday, Деспро — стр...",0,0.788404,0.864865,0.800000,0.864865,19,18


# **RuGPT model test**

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import AutoPeftModelForCausalLM

from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.tokenize import word_tokenize
from rouge import Rouge
import nltk
nltk.download('punkt')

# Количество примеров
N = 50

# Загрузка тестового датасета
test_data = pd.read_csv('test_dataset.csv')
originals_df = test_data[['original']].head(N)
references_df = test_data[['replaced']].head(N)

original = originals_df['original'].tolist()
references = references_df['replaced'].tolist()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Функция загрузки модели
def load_model1():
    model = AutoPeftModelForCausalLM.from_pretrained("nata2627/angl_detection_tokenizer_ruGPT")
    tokenizer = AutoTokenizer.from_pretrained("nata2627/angl_detection_tokenizer_ruGPT")

    model = model.to("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    return model, tokenizer

In [ ]:
def cut_after_first_dot(text):
    """
    Оставляет текст до первой точки (включительно), если она есть.
    """
    idx = text.find('.')
    return text[:idx + 1] if idx != -1 else text

def generate_predictions(model, tokenizer, prompts):
    predictions = []
    device = next(model.parameters()).device
    for prompt in prompts:
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=100,
                do_sample=False,
                temperature=0.7,
                top_p=0.9,
                top_k=50,
                no_repeat_ngram_size=2
            )
        decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
        generated_text = decoded.replace(prompt, "").strip()
        final_output = cut_after_first_dot(generated_text)
        predictions.append(final_output)
    return predictions


In [ ]:
# Загрузка модели и токенизатора
model1, tokenizer1 = load_model1()

In [ ]:
predictions = generate_predictions(model1, tokenizer1, original)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [ ]:
predictions

['По словам женщины, лифты в доме часто ломаются, она уже оставила жалобу в обслуживающей компании и планирует обратиться в правоохранительные органы.',
 'Об этом официальный представитель МИД России Мария Захарова сказала во время еженедельного брифинга.',
 '«Глубина бронирования остается на прежнем уровне, это природное явление, которое очень редко бывает в Эмиратах, поэтому проблема будет решена за один-два дня»,\xa0— подытожил вице-президент РСТ.',
 'Сафиуллин сейчас занимает 42-е место в рейтинге ATP (четвертый из россиян).',
 'В последний раз Смоленская область подверглась атаке дронов 24 апреля, на топливно-энергетических объектах в Смоленском и Ярцевском районах начались пожары.',
 'Тройчук будет руководителем проекта, у которого есть инвестор.',
 'Интернет-портал «Чемпионат» 8 апреля сообщил, что голландец скоро возглавит московский клуба.',
 'Обвиняемыми по делу проходят более 30 человек, в том числе директор филиала «Дагэнерго»,\xa0одного из крупнейших подразделений «Россети

In [ ]:
# Метрики оценки
smoothie = SmoothingFunction().method4
rouge = Rouge()

metrics_table = []

for pred, ref in zip(predictions, references):
    exact_match = int(pred == ref)

    pred_tokens = word_tokenize(pred.lower())
    ref_tokens = [word_tokenize(ref.lower())]
    bleu = sentence_bleu(ref_tokens, pred_tokens, smoothing_function=smoothie)

    rouge_score = rouge.get_scores(pred, ref)[0]

    metrics_table.append({
        'prediction': pred,
        'reference': ref,
        'exact_match': exact_match,
        'bleu': bleu,
        'rouge-1': rouge_score['rouge-1']['f'],
        'rouge-2': rouge_score['rouge-2']['f'],
        'rouge-l': rouge_score['rouge-l']['f'],
        'prediction_length': len(pred.split()),
        'reference_length': len(ref.split())
    })

# Сохранение и вывод
df1 = pd.DataFrame(metrics_table)
df1.to_csv('rugpt_metrics_table.csv', index=False)

summary_metrics1 = {
    'exact_match_accuracy': df['exact_match'].mean(),
    'bleu_score': df['bleu'].mean(),
    'rouge-1': df['rouge-1'].mean(),
    'rouge-2': df['rouge-2'].mean(),
    'rouge-l': df['rouge-l'].mean(),
    'avg_prediction_length': df['prediction_length'].mean(),
    'avg_reference_length': df['reference_length'].mean()
}

# Преобразуем сводные метрики в DataFrame
summary_df1 = pd.DataFrame([summary_metrics1])
summary_df1.to_excel('rugpt_summary_metrics.xlsx', index=False)

print(df1)
print(summary_metrics1)

                                           prediction  \
0   По словам женщины, лифты в доме часто ломаются...   
1   Об этом официальный представитель МИД России М...   
2   «Глубина бронирования остается на прежнем уров...   
3   Сафиуллин сейчас занимает 42-е место в рейтинг...   
4   В последний раз Смоленская область подверглась...   
5   Тройчук будет руководителем проекта, у которог...   
6   Интернет-портал «Чемпионат» 8 апреля сообщил, ...   
7   Обвиняемыми по делу проходят более 30 человек,...   
8   Атаки украинских дронов на Запорожскую АЭС — у...   
9   В то же время, по данным WAtoday, Деспро — стр...   
10  Украина нужна Европейскому союзу и станет хоро...   
11  О взятии под контроль последнего военное ведом...   
12  Он был главой и совладельцем немецкой сети суп...   
13  Постепенно его распространяют и на другие субъ...   
14  Главный тренер «Ростова» Валерий Карпин пропус...   
15  На территории двух государств также расположен...   
16  НАТО не пригласит Украину в

# **Qwen model test**

In [1]:
!pip install rouge

In [27]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import AutoPeftModelForCausalLM

from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.tokenize import word_tokenize
from rouge import Rouge
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

# Количество примеров
N = 50

# Загрузка тестового датасета
test_data = pd.read_csv('test_dataset.csv')
originals_df = test_data[['original']].head(N)
references_df = test_data[['replaced']].head(N)

original = originals_df['original'].tolist()
references = references_df['replaced'].tolist()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [8]:
# Функция загрузки модели
def load_model2():
    model = AutoPeftModelForCausalLM.from_pretrained(
        "nata2627/angl_detection_tokenizer_qwen",
        device_map="auto",
        torch_dtype=torch.float16,
        ignore_mismatched_sizes=True
    )
    tokenizer = AutoTokenizer.from_pretrained("nata2627/angl_detection_tokenizer_qwen")

    model = model.to("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    return model, tokenizer

In [9]:
# Загрузка модели и токенизатора
model2, tokenizer2 = load_model2()

/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:470: UserWarning: Some weights of PeftModelForCausalLM were not initialized from the model checkpoint and are being ignored because you passed `ignore_mismatched_sizes=True`: - base_model.model.lm_head.weight: found shape torch.Size([151667, 1536]) in the checkpoint and torch.Size([151936, 1536]) in the model instantiated
- base_model.model.model.embed_tokens.weight: found shape torch.Size([151667, 1536]) in the checkpoint and torch.Size([151936, 1536]) in the model instantiated.
  warnings.warn(msg)


In [28]:
# Генерация предсказаний с правильным форматированием промпта
predictions = []
for input_text in original:
    # Создаем инструкцию для модели (как во время обучения)
    instruction = (
        "Инструкция: Замените англицизмы в тексте на их русские аналоги.\n\n"
        f"Текст: {input_text}\n\n"
        f"Результат:"
    )

    # Форматируем промпт как во время обучения
    formatted_input = f"<|im_start|>user\n{instruction}<|im_end|>\n<|im_start|>assistant\n"

    # Токенизация
    inputs = tokenizer2(formatted_input, return_tensors="pt").to(model2.device)

    # Генерация с теми же параметрами, что и в обучении
    with torch.no_grad():
        outputs = model2.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer2.eos_token_id
        )

    # Декодирование и извлечение только ответа ассистента
    pred_text = tokenizer2.decode(outputs[0], skip_special_tokens=False)
    assistant_part = pred_text.split("<|im_start|>assistant\n")[-1].split("<|im_end|>")[0]
    predictions.append(assistant_part.strip())

In [29]:
predictions

['По словам женщины, подъемник в доме часто ломается, она уже оставила жалобу в обслуживающей компании и планирует обратиться в правоохранительные органы.',
 'Об этом официальный представитель МИД России Мария Захарова сказала во время еженедельной встречи.',
 '«Глубина бронирования остается на прежнем уровне, это природное явление, которое очень редко бывает в Эмиратах, поэтому проблема будет решена за один-два дня», — подытожил заместитель президента РСТ.',
 'Сафиуллин сейчас занимает 42-е место в оценке ATP (четвертое из россиян).',
 'В последний раз Смоленская область подверглась атаке беспилотником 24 апреля, на топливно-энергетических объектах в Смоленском и Ярцевском районах начались пожары.',
 'Тройчук будет руководителем проекта, у которого есть вкладчик средств.',
 'Интернет-портал «Чемпионат» 8 апреля сообщил, что голландец скоро возглавит московское подразделение.',
 'Обвиняемыми по делу проходят более 30 человек, в том числе руководитель филиала «Дагэнерго», одного из круп

In [30]:
# Метрики оценки
smoothie = SmoothingFunction().method4
rouge = Rouge()

metrics_table = []

for pred, ref in zip(predictions, references):
    exact_match = int(pred == ref)

    pred_tokens = word_tokenize(pred.lower())
    ref_tokens = [word_tokenize(ref.lower())]
    bleu = sentence_bleu(ref_tokens, pred_tokens, smoothing_function=smoothie)

    rouge_score = rouge.get_scores(pred, ref)[0]

    metrics_table.append({
        'prediction': pred,
        'reference': ref,
        'exact_match': exact_match,
        'bleu': bleu,
        'rouge-1': rouge_score['rouge-1']['f'],
        'rouge-2': rouge_score['rouge-2']['f'],
        'rouge-l': rouge_score['rouge-l']['f'],
        'prediction_length': len(pred.split()),
        'reference_length': len(ref.split())
    })

# Сохранение и вывод
df = pd.DataFrame(metrics_table)
df.to_csv('qwen_metrics_table.csv', index=False)

summary_metrics = {
    'exact_match_accuracy': df['exact_match'].mean(),
    'bleu_score': df['bleu'].mean(),
    'rouge-1': df['rouge-1'].mean(),
    'rouge-2': df['rouge-2'].mean(),
    'rouge-l': df['rouge-l'].mean(),
    'avg_prediction_length': df['prediction_length'].mean(),
    'avg_reference_length': df['reference_length'].mean()
}

# Преобразуем сводные метрики в DataFrame
summary_df = pd.DataFrame([summary_metrics1])
summary_df.to_excel('qwen_summary_metrics.xlsx', index=False)

print(df)
print(summary_metrics)

                                           prediction  \
0   По словам женщины, подъемник в доме часто лома...   
1   Об этом официальный представитель МИД России М...   
2   «Глубина бронирования остается на прежнем уров...   
3   Сафиуллин сейчас занимает 42-е место в оценке ...   
4   В последний раз Смоленская область подверглась...   
5   Тройчук будет руководителем проекта, у которог...   
6   Интернет-портал «Чемпионат» 8 апреля сообщил, ...   
7   Обвиняемыми по делу проходят более 30 человек,...   
8   Атаки украинских беспилотников на Запорожскую ...   
9   В то же время, по данным WAtoday, Деспро — стр...   
10  Украина нужна Европейскому союзу и станет хоро...   
11  О взятии под проверку последнего военное ведом...   
12  Он был главой и совладельцем немецкой сети кру...   
13  Постепенно его распространяют и на других членов.   
14  Главный спортивный наставник «Ростова» Валерий...   
15  На территории двух государств также расположен...   
16  НАТО не пригласит Украину в